In [1]:
import os
import re
import pandas as pd
from langchain_openai import ChatOpenAI
from langchain_community.graphs import Neo4jGraph


os.environ["OPENAI_API_KEY"] = "sk-"

llm = ChatOpenAI(model_name="gpt-4-turbo", request_timeout=180)
DEMO_URL = "neo4j+s://demo.neo4jlabs.com"

In [3]:
# Load the JSONL file into a DataFrame
df = pd.read_json("text2cypher_batch_output.jsonl", lines=True)
df

,id,custom_id,response,error
0,batch_req_O0iYejkKUIm6PySDuP16SxOC,request-0,"{'status_code': 200, 'request_id': 'f6cdf9300c...",NaN
1,batch_req_Vi2n7Z6o9c1luzKoV1FTE1Bd,request-1,"{'status_code': 200, 'request_id': 'c9ded535e5...",NaN
2,batch_req_WLQwdwd23ZecEiCM4ZCxw6mk,request-2,"{'status_code': 200, 'request_id': 'ff25238d19...",NaN
3,batch_req_q6lB2BIxq1VcSCKxzaAevu4W,request-3,"{'status_code': 200, 'request_id': 'eab91ac91f...",NaN
4,batch_req_YKy7DWW8b7gCi4ZySgeZIwNJ,request-4,"{'status_code': 200, 'request_id': '2f4e70c3f5...",NaN
...,...,...,...,...
9819,batch_req_63Ofjq5l6Lecs3XLLgn1EjGV,request-9819,"{'status_code': 200, 'request_id': '017c608a8d...",NaN
9820,batch_req_bCG36LwfGPmY6ggtZN9zSqxf,request-9820,"{'status_code': 200, 'request_id': '38723b5f3f...",NaN
9821,batch_req_Y8OJIPS2s6IBENAtSWZYsdbT,request-9821,"{'status_code': 200, 'request_id': '67b17a4454...",NaN
9822,batch_req_b4gtfaiDPA2ABb45hQq88YB4,request-9822,"{'status_code': 200, 'request_id': '9f10be8911...",NaN


In [21]:
def extract_text(text):
    # Adjust pattern to capture after ```cypher and spans multiple lines until ```
    pattern = r"```cypher\n(.*?)\n```"
    match = re.search(pattern, text, re.DOTALL)

    if match:
        # Return the extracted text if triple backticks are present
        return match.group(1).strip()
    else:
        # Return the original text if triple backticks are not present
        return text


df["cypher"] = [
    extract_text(el["body"]["choices"][0]["message"]["content"])
    for el in df["response"]
]

In [27]:
meta_df = pd.read_csv("text2cypher_questions.csv")
meta_df["id"] = [f"request-{el}" for el, _ in enumerate(meta_df["database"])]
meta_df.head()

,question,type,database,id
0,What are the top 5 movies with a runtime great...,Simple Retrieval Queries,recommendations,request-0
1,List the first 3 directors born before 1950.,Simple Retrieval Queries,recommendations,request-1
2,Which 5 users have rated more than 20 movies?,Simple Retrieval Queries,recommendations,request-2
3,Identify the top 5 actors who have acted in mo...,Simple Retrieval Queries,recommendations,request-3
4,What are the top 3 genres associated with movi...,Simple Retrieval Queries,recommendations,request-4


In [32]:
combined_df = pd.merge(df, meta_df, left_on="custom_id", right_on="id")
combined_df = combined_df.sort_values(by="database")
combined_df.head()

,id_x,custom_id,response,error,cypher,question,type,database,id_y
483,batch_req_mH0DRhUiLp0OpOrSINn1tDXW,request-483,"{'status_code': 200, 'request_id': 'b5547bfdd8...",NaN,MATCH (u:User)-[r:INTERACTED]->(:User)\nWHERE ...,Who are the top 5 users with the most interact...,Complex Retrieval Queries,bluesky,request-483
487,batch_req_b8rCrVJYbwnq8hnZNjsPOrLb,request-487,"{'status_code': 200, 'request_id': 'e11dbacb98...",NaN,MATCH (u:User)-[r:INTERACTED]->(:User)\nWHERE ...,List the top 5 users having the most interacti...,Complex Retrieval Queries,bluesky,request-487
486,batch_req_uVxhemAnwHga6d1eWP9526nm,request-486,"{'status_code': 200, 'request_id': 'f7bd6ed44c...",NaN,MATCH (u:User)\nWHERE u.color = '#3CB371' AND ...,Which 3 users have the largest size and a colo...,Complex Retrieval Queries,bluesky,request-486
485,batch_req_6JRQTx9yL2j9R4vboWKhy7gY,request-485,"{'status_code': 200, 'request_id': '72e8aaa86f...",NaN,MATCH (u:User)-[:INTERACTED]->(v:User {color: ...,Show the top 5 users who have interacted with ...,Complex Retrieval Queries,bluesky,request-485
484,batch_req_juKdpCRV7Itlm8u8j9tDmDIp,request-484,"{'status_code': 200, 'request_id': '165dc179bd...",NaN,MATCH (u:User {label: 'sarahp.bsky.social'})-[...,Find the top 3 users interacted with by 'sarah...,Complex Retrieval Queries,bluesky,request-484


In [ ]:
syntax_error = []
returns_results = []
timeouts = []
last_graph = ""
for i, row in combined_df.reset_index().iterrows():
    if i % 100 == 0:
        print(i)
    # To avoid a new driver for every request
    if row["database"] != last_graph:
        last_graph = row["database"]
        print(last_graph)
        graph = Neo4jGraph(
            url=DEMO_URL,
            username=row["database"],
            password=row["database"],
            database=row["database"],
            refresh_schema=False,
            timeout=10,
        )
    try:
        data = graph.query(row["cypher"])
        if data:
            returns_results.append(True)
        else:
            returns_results.append(False)
        syntax_error.append(False)
        timeouts.append(False)
    except ValueError as e:
        if "Generated Cypher Statement is not valid" in str(e):
            syntax_error.append(True)
            print(f"Syntax error in Cypher query: {e}")
        else:
            syntax_error.append(False)
            print(f"Other ValueError: {e}")
        returns_results.append(False)
        timeouts.append(False)
    except Exception as e:
        if (
            e.code
            == "Neo.ClientError.Transaction.TransactionTimedOutClientConfiguration"
        ):
            returns_results.append(False)
            syntax_error.append(False)
            timeouts.append(True)

In [52]:
combined_df["syntax_error"] = syntax_error
combined_df["timeout"] = timeouts
combined_df["returns_results"] = returns_results

In [62]:
final_df = combined_df[
    [
        "question",
        "cypher",
        "type",
        "database",
        "syntax_error",
        "timeout",
        "returns_results",
    ]
]
final_df.head()

,question,cypher,type,database,syntax_error,timeout,returns_results
483,Who are the top 5 users with the most interact...,MATCH (u:User)-[r:INTERACTED]->(:User)\nWHERE ...,Complex Retrieval Queries,bluesky,False,False,True
487,List the top 5 users having the most interacti...,MATCH (u:User)-[r:INTERACTED]->(:User)\nWHERE ...,Complex Retrieval Queries,bluesky,False,False,False
486,Which 3 users have the largest size and a colo...,MATCH (u:User)\nWHERE u.color = '#3CB371' AND ...,Complex Retrieval Queries,bluesky,False,False,True
485,Show the top 5 users who have interacted with ...,MATCH (u:User)-[:INTERACTED]->(v:User {color: ...,Complex Retrieval Queries,bluesky,False,False,True
484,Find the top 3 users interacted with by 'sarah...,MATCH (u:User {label: 'sarahp.bsky.social'})-[...,Complex Retrieval Queries,bluesky,False,False,True


In [63]:
# Assume df is your DataFrame and col1, col2, col3 are the boolean columns
distribution_col1 = final_df["syntax_error"].value_counts()
distribution_col2 = final_df["timeout"].value_counts()
distribution_col3 = final_df["returns_results"].value_counts()

print("Distribution for syntax_error:\n", distribution_col1)
print("Distribution for timeout:\n", distribution_col2)
print("Distribution for returns_results:\n", distribution_col3)

Distribution for syntax_error:
 syntax_error
False    9537
True      287
Name: count, dtype: int64
Distribution for timeout:
 timeout
False    9664
True      160
Name: count, dtype: int64
Distribution for returns_results:
 returns_results
True     7685
False    2139
Name: count, dtype: int64


In [64]:
final_df.to_csv("text2cypher_gpt4turbo.csv", index=False)